In [60]:
%reset -f
import torch
from torchvision import transforms
from matplotlib import pyplot as plt
from models import TCNmodel
import numpy as np
from random import randrange

train_sample = torch.load('tensor.pt')
train_sample=train_sample.transpose(1,2)
#print(train_data.shape)
preprocess = transforms.Compose([
    #transforms.Resize([41,360,640]),
    #transforms.CenterCrop(299),
    #transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
train_sample = preprocess(train_sample)
print(train_sample.shape)

#data_loader = torch.utils.data.DataLoader(train_sample, batch_size = 1, shuffle = True)
# for videos in data_loader:
#     print(videos.size())

torch.Size([1, 41, 3, 360, 640])


In [61]:
def DatasetCon(data):
    # construct a F x 3 x C x W x H tensoR
    # 0: anchor, 1: positive, 2: negative
    N,F,C,W,H = data.shape # F: frame
    anchor = data
    idx = np.arange(F)

    # positve & negative
    posindex = np.zeros(F)
    negindex = np.zeros(F)
    for i in range(F):
        # pp positve
        pp= randrange(-1,1)
        if pp ==-1:
            posindex[i]=-1
        else:
            posindex[i]=1
        
        # pn negative      
        pn = randrange(0,F)
        while pn<=i+2 & pn>=i-2:
            pn=randrange(0,F)
        negindex[i]=pn
    
    # fixed index value at 0 frame and end frame
    posindex[0]=1
    posindex[-1]=-1
    posindx = idx+posindex
    positive = data[:,posindex,:,:,:]

    negative = data[:,negindex,:,:,:]
    dataset = torch.cat([anchor,positive,negative])
    dataset = dataset.transpose(0,1)
    return dataset

In [62]:
# dataset = DatasetCon(train_sample)
# print(dataset.shape)
# data_loader = torch.utils.data.DataLoader(dataset, batch_size = 4, shuffle = True)
# for videos in data_loader:
#     print(videos.size())
#     break


In [63]:
# model=TCNmodel()

In [64]:
# with torch.no_grad():
#     output = model(torch.reshape(videos,[-1,3,360,640]))
# # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output.shape)

In [65]:
# output=output.reshape([4,3,-1])
# print(output.shape)
# anchor = output[:,0,:]
# positive = output[:,1,:]
# negative = output[:,2,:]
# triplet_loss = torch.nn.TripletMarginLoss(margin=0.2, p=2)
# # corr=torch.mm(output,output.transpose(0,1))
# loss = triplet_loss(anchor, positive, negative)
# print(loss)


In [66]:
def train(train_loader, net, optimizer, criterion,device):
    """
    Trains network for one epoch in batches.

    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        criterion: Loss function (e.g. cross-entropy loss).
    """
  
    avg_loss = 0
    #correct = 0
    total = 0

    # iterate through batches
    for i, data in enumerate(train_loader):
        N,P,C,W,H = data.shape
        # get the inputs; data is a list of [inputs, labels]
        inputs = data.reshape([-1,C,W,H])
        inputs= inputs.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        outputs = outputs.reshape([N,P,-1])
        
        anchor = outputs[:,0,:]
        positive = outputs[:,1,:]
        negative = outputs[:,2,:]
        
        loss = criterion(anchor,positive,negative)
        loss.backward()
        optimizer.step()

        # keep track of loss and accuracy
        avg_loss += loss
        #print(avg_loss)
    return avg_loss/len(train_loader)

In [67]:
from tqdm import tqdm
epochs = 10

# Create instance of Network
net = TCNmodel()

# Create loss function and optimizer
criterion = torch.nn.TripletMarginLoss(margin=0.2, p=2)
optimizer = torch.optim.SGD(net.parameters(), lr=5e-1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
net = net.to(device)
# print(device)
for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    # randomly generate pair dataset
    dataset = DatasetCon(train_sample)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size = 4, shuffle = True)
    # Train on data
    train_loss = train(data_loader,net,optimizer,criterion,device)
    #print(train_loss)
    torch.cuda.empty_cache()

Using cache found in C:\Users\Think/.cache\torch\hub\pytorch_vision_v0.10.0
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:52<00:00,  5.24s/it]
